<center><h1><b>Trabalho Computacional 02</b></h1></center>

Autor: Rômulo Bandeira Pimentel Drumond

<h2>
1. Questão 01
</h2>

Utilizando o conjunto de dados *Arcene* (disponível em: https://archive.ics.uci.edu/ml/datasets/Arcene ), três modelos de classificação foram implementados:

* Mínimos Quadrados Linear (MQL);
* Perceptron Simples (PS);
* Perceptron Multicamadas (MLP).

Cada um deles foi tratado em detalhes nos tópicos a seguir.

<h3>
1.1 Mínimos Quadrados Linear
</h3>

<h4>
1.1.1 Pré-processamento dos dados
</h4>

Primeiramente o conjunto de dados foi carregado no sistema computacional. Como a metodologia adotada para treinar e testar os modelos é aplicar cem vezes uma separação aleatória do *dataset* completo entre dois grupos, o conjunto de dados que antes estava dividido entre treino e validação foi unificado utilizando o código abaixo.

In [1]:
import pandas as pd

trainData   = pd.read_csv('dataset/arcene_train.data', delim_whitespace=True, header=None)
trainLabels = pd.read_csv('dataset/arcene_train.labels', delim_whitespace=True, names=['Classe'])

validData   = pd.read_csv('dataset/arcene_valid.data', delim_whitespace=True, header=None)
validLabels = pd.read_csv('dataset/arcene_valid.labels', delim_whitespace=True, names=['Classe'])

/home/romulo/Dropbox/Mestrado/[00 old]/[DISCIPLINA] Inteligência Computacional Aplicada/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
datasetX = pd.concat([trainData, validData])     # entradas do sistema
datasetX = datasetX.reset_index(drop=True) # reorganizando os índices

datasetY = pd.concat([trainLabels, validLabels]) # saídas do sistema
datasetY = datasetY.reset_index(drop=True) # reorganizando os índices

dataset = pd.concat([datasetX, datasetY], axis=1)

print("Estatísticas do dataset:\n\
Número de amostras: {}\n\
Número de features: {}\n\
Número de saídas: {}".format(datasetX.shape[0], datasetX.shape[1], datasetY.shape[1]))

Estatísticas do dataset:
Número de amostras: 200
Número de features: 10000
Número de saídas: 1


Algumas estatísticas do *dataset* foram calculadas:

In [3]:
%%time
statsDataset = dataset.describe()
display(statsDataset)

,0,1,2,3,4,5,6,7,8,9,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,Classe
count,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,...,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,35.385000,37.055000,60.305000,325.425000,17.055000,410.940000,226.310000,7.695000,48.795000,70.765000,...,383.355000,46.150000,18.475000,107.170000,177.870000,202.205000,0.690000,87.965000,361.650000,-0.120000
std,48.755742,37.954905,71.168439,155.169819,27.095068,105.681404,181.533678,14.893659,58.164927,96.738469,...,234.985489,48.948619,24.981186,157.800399,142.621025,118.424172,3.367581,111.476224,104.970336,0.995265
min,0.000000,0.000000,0.000000,0.000000,0.000000,181.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000
25%,0.000000,0.750000,0.000000,211.500000,0.000000,311.750000,0.000000,0.000000,0.000000,0.000000,...,236.250000,0.000000,0.000000,0.000000,0.000000,92.750000,0.000000,0.000000,312.750000,-1.000000
50%,10.500000,27.500000,23.500000,334.000000,0.000000,431.500000,245.500000,0.000000,25.500000,8.000000,...,459.000000,37.000000,0.000000,44.500000,214.500000,183.500000,0.000000,38.500000,361.000000,-1.000000
75%,54.000000,59.250000,116.000000,433.750000,29.500000,497.000000,410.250000,9.000000,79.250000,143.250000,...,574.250000,81.000000,35.250000,130.000000,283.250000,314.500000,0.000000,159.250000,446.000000,1.000000
max,194.000000,218.000000,240.000000,705.000000,109.000000,635.000000,501.000000,74.000000,202.000000,425.000000,...,692.000000,200.000000,90.000000,656.000000,509.000000,423.000000,24.000000,491.000000,533.000000,1.000000


CPU times: user 20.6 s, sys: 133 ms, total: 20.7 s
Wall time: 20.7 s


Devido a heterogeniedade dos valores das variáveis de entrada foi aplicada a normalização a seguir:

$$u_i=\frac{x_i-\hat{\mu}_i}{\hat{\sigma}_i}$$

Essa operação foi realizada para, mais a frente, tornar transparente a influência de cada variável no valor predito pelo modelo (no caso do MQL).

In [4]:
# Normalização das variáveis de entrada
datasetX_normal = datasetX.copy()
datasetX_normal = (datasetX_normal - datasetX_normal.mean()) / datasetX_normal.std()

Durante a regressão linear múltipla ocorreu um erro: havia no conjunto de dados normalizado valores *'NaN'*, possivelmente causados por algum erro numérico durante a normalização. A primeira hipótese proposta foi a existência de variáveis de entrada $\hat{\sigma}_i=0$, para averiguar isso foi feita uma busca por variáveis com desvio-padrão nulo.

In [5]:
std = datasetX.std()
indices = std.index[std == 0].tolist()
print("Variáveis de entrada com desvio-padrão nulo: {}".format(indices))
print("\nNúmero de variáveis com desvio-padrão nulo: {}".format(len(indices)))

Variáveis de entrada com desvio-padrão nulo: [113, 215, 557, 592, 993, 1956, 2349, 2482, 2520, 2583, 3020, 3273, 3390, 3562, 3657, 3714, 4234, 4238, 4621, 4637, 4649, 4676, 4846, 4847, 5034, 5415, 5626, 5735, 5756, 5940, 6346, 6995, 7091, 7809, 8704, 8840, 8878, 9538, 9913]

Número de variáveis com desvio-padrão nulo: 39


Foram analisadas as estatísticas dessas variáveis, visíveis na Tabela \ref{tab:statsDataset}:

In [6]:
statsDataset[indices] # tabela de estatísticas das variáveis com desvio-padrão nulo

,113,215,557,592,993,1956,2349,2482,2520,2583,...,5940,6346,6995,7091,7809,8704,8840,8878,9538,9913
count,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,...,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0,200.0
mean,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
max,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Claramente essas variáveis encontradas são identicamente nulas, contendo nenhum poder preditivo no modelo. Tais entradas fazem parte do conjunto de  variáveis \textit{probe} que foram inseridas propositalmente no \textit{dataset} para dificultar a modelagem (o conjunto de dados \textit{Arcene} fez parte de uma competição do NIPS em 2003). Logo as 39 variáveis foram retiradas.

In [7]:
datasetX_normal2 = datasetX_normal.drop(indices, 1)
print("Número de variáveis restantes: {}.".format(datasetX_normal2.shape[1]))

Número de variáveis restantes: 9961.


Procurou-se também por variáveis repetidas no conjunto de dados. O código a seguir aplica essa busca:

In [8]:
# Identificação de variáveis repetidas
Xv3 = datasetX_normal2.transpose() # a transposta é feita visto que o método identificar linhas repetidas, não colunas
indices1 = Xv3.duplicated()[Xv3.duplicated()].index.values # índices de variáveis repetidas
print("Variáveis repetidas: {} \nNúmero de variáveis repetidas: {}".format(indices1, len(indices1)))

Variáveis repetidas: [ 436  466  652 5085 5313 5348 5517 5610 6762 7106 7447 8320 8934 9895] 
Número de variáveis repetidas: 14


Tais variáveis repetidas foram retiradas do \textit{dataset}:

In [9]:
X_v3 = datasetX_normal2.drop(indices1, 1)
print("Número de variáveis restantes: {}.".format(X_v3.shape[1]))

Número de variáveis restantes: 9947.


Conclui-se o pré-processamento dos dados com 9.947 variáveis de entrada, restando portanto 2.947 variáveis \textit{probes}.

<h4>
1.1.2 Identificação de variáveis *probe* através de regressão linear múltipla
</h4>

Como ainda nos restaram 2.947 variáveis *probes* foi necessário aplicar outra metodologia para identificá-las. Algumas propostas foram:

* Analisar performance dos modelos para todas as combinações possíves de variáveis de entrada, no entanto, o número de combinações se torna $2^{9947} \approx 10^{2994}$, o que é inviável;
* Analisar as possíveis combinações de 7.000 variáveis do conjunto das 9.947, mas esse número também é imenso $\binom{9947}{7000} \approx 10^{2623}$;
* *Stepwise regression* \cite{stepwise}:
  * *Forward selection*: adicionar as variáveis mais importantes, uma a uma, até escolhermos 7.000;
  * *Backward elimination*: retirar variáveis menos importantes, uma a uma, até retirarmos 2.947.
  
Foi escolhida a *stepwise regression* com *backward elimination*, a motivação principal foi a menor complexidade computacional (poder e tempo de computação) envolvida. Com o tempo de processamente em mente a implementação da regressão foi feita utilizando a biblioteca *scikit-learn*, por essa possuir métodos otimizados para a solução de MQL.

Para ter uma referência de tempo de execução foi feita uma regressão utilizando os 9.947 atributos restantes:

In [10]:
%%time
# Comparativo com scikit-learn:
from sklearn import linear_model
reg = linear_model.LinearRegression()
Y = datasetY # unificando a nomenclatura
reg.fit(X_v3.values, Y.values)

CPU times: user 1.48 s, sys: 1.55 s, total: 3.03 s
Wall time: 2.14 s


Sabendo que numa regressão linear, com variáveis de entrada normalizadas, os coeficientes multiplicativos são um indicativo da influência da variável no modelo, foram feitos o histograma (Figura \ref{fig:histoModCoef}) e boxplot (Figura \ref{fig:boxModCoef}) do módulo dos coeficientes. O código abaixo aplica o que foi proposto nesse parágrafo:

In [11]:
import plotly.offline as plt
import plotly.graph_objs as go

plt.init_notebook_mode(connected=True) # habilitando plot dentro do jupyter notebook

# Histograma dos módulos dos coeficientes
def plotHistogram(x_vec):
    data = [go.Histogram(
        x=x_vec,
        histnorm="probability density",
    )]

    layoutHist = go.Layout(
        title='Histograma do módulo dos coeficientes',
        xaxis=dict(title='Módulo do coeficiente'),
        yaxis=dict(title='Densidade de probabilidade')
    )

    fig = go.Figure(data=data, layout=layoutHist)
    plt.iplot(fig)

# Boxplot do módulo dos coeficientes
def plotBoxplot(x_vec):
    data = [
        go.Box(
            y= x_vec,
            boxpoints='all',
            boxmean='sd',
            jitter=0.3,
            pointpos=-1.8
        )
    ]
    
    layoutBoxplot = go.Layout(title="Boxplot do módulo dos coeficientes")

    fig = go.Figure(data=data, layout=layoutBoxplot)
    plt.iplot(fig)

w_abs = abs(reg.coef_[0]) # módulo dos coeficientes da regressão
plotHistogram(w_abs)
plotBoxplot(w_abs)

Nas Figuras \ref{fig:histoModCoef}) e \ref{fig:boxModCoef} grande parte dos coeficientes tem módulos muito próximos de zero, uma evidência da existência das outras variáveis *probes*.

Para o contexto desse trabalho, um modelo performante será o que tiver maior taxa de acerto no conjunto de teste, visto que se trata de um problema de classificação. A taxa de acerto no *dataset* de teste é uma boa métrica por tentar evitar o *undefitting* e *overfitting* do modelo (supondo que o conjunto de dados seja uma boa representação do fenômeno estudado). Entretanto, a taxa de acerto no conjunto de teste ($t_s$) é uma variável aleatória, visto que a separação do *dataset* entre treino e teste será repetida 100 vezes para cada configuração de variáveis de entrada. Na realidade o processo de identificação das variáveis menos importantes pode ser traduzido num problema de otimização \cite{boyd}:

$$\text{maximizar } f_0(ts)=\mathbb{E}\{t_s\} - \gamma\sigma\{t_s\}$$

Ou seja, busca-se capacidade de generalização do modelo e estabilidade. Sabendo que $\gamma \geq 0$ ela é a variável que busca o *trade-off* entre os dois objetivos:

* Maximizar esperança da taxa de acerto no conjunto de teste, ou seja, melhorar a generalização;
* Minimizar o desvio-padrão da taxa de acerto do conjunto de teste, ou seja, melhorar a estabilidade.

O valor de adotado foi $\gamma=2$.

Como retirar as variáveis uma a uma se torna um processo pesado devido à quantidade de variáveis (teriam que ser relizadas $100 \cdot 9947!/7000! \approx 10^{11571}$ regressões), um *script* foi criado para avaliar $f_0$ em cada uma das 9.947 variáveis e retirar as 2.947 menos performantes (sendo necessário "apenas" $100 \cdot 9947 \approx 10^6$ regressões). O código abaixo implementou essa avaliação.

In [12]:
import numpy as np

# Função sinal para ser aplicada na saída
def sinal(x):
    return +1 if x >= 0 else -1

vSinal  = np.vectorize(sinal) # Versão vetorial da função sinal

In [13]:
resultado = pd.read_csv('resultado.csv')
resultado = resultado.sort_values("f_0(ts)", ascending=False)
resultado.head()

,Variável,Mean(ts),Std(ts),f_0(ts)
9400,9450,0.89975,0.038323,0.823103
6514,6553,0.89350,0.037686,0.818128
7307,7350,0.89500,0.038891,0.817218
5694,5729,0.89775,0.040326,0.817098
5304,5334,0.90100,0.041970,0.817060


Os 2.947 modelos mais performantes são aqueles que a retirada da respectiva variável é menos impactante, consequentemente retirou-se essas variáveis:

In [14]:
badVar = resultado["Variável"].values[0:2947]
badVar = np.sort(badVar)

X_v4 = X_v3.drop(badVar, 1)
print("Variáveis restantes: {}.".format(X_v4.shape[1]))

Variáveis restantes: 7000.


Refazendo a análise dos módulos dos pesos para o conjunto de dados restantes:

In [15]:
reg.fit(X_v4.values, Y.values)
w_abs = abs(reg.coef_[0]) # módulo dos coeficientes da regressão
plotHistogram(w_abs)
plotBoxplot(w_abs)

Houve um pequeno aumento da dispersão, favorecendo a hipótese de que as variáveis retiradas eram falsas. Embora 7.000 varíaveis de entrada para 200 amostras não seja uma boa relação, por propiciar o \textit{overfitting} do modelo, essas variáveis foram mantidas pois a retirada das últimas 2.947 não foi feita de forma muito criteriosa. É importante ressaltar que as relações entre as variáveis não foi estudada a fundo e é possível que na última poda tenham sido eliminadas entradas importantes para o modelo, por isso a redução do número de variáveis de entrada para por aqui.

<h4>
1.1.3 Avaliação do modelo linear
</h4>

A metodologia adotada para avaliar os modelos foi:
1. Separar aleatoriamente 80% do *dataset* para treinar o classificador e os 20% restantes foram usados para testar;
2. Repetiu-se 100 vezes o processo de separação dos dados em conjuntos de treinamento e teste e armazenou-se a taxa de acerto para cada uma das 100 rodadas.
3. Após o término das 100 rodadas de treinamento e teste, calculou-se o desvio-padrão e as taxas de acerto média, mínima e máxima;
4. Calculou-se as taxas de acerto médias por classe.

In [16]:
%%time
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

tr = [0]*100 # taxa de acerto no treino
ts = [0]*100 # taxa de acerto no teste
cm_tr = np.zeros((2,2)) # matriz de confusão da média no treino
cm_ts = np.zeros((2,2)) # matriz de confusão da média no teste
for i in range(100):
    # Separação aleatória do dataset em 80% de treino e 20% de teste
    X_tr, X_ts, Y_tr, Y_ts = train_test_split(X_v4.values, Y.values, test_size=0.2)
    
    # Regressão linear com conjunto de treino;
    reg.fit(X_tr, Y_tr)
    a, b = reg.coef_[0], reg.intercept_[0] # coeficientes da regressão

    # Calculando taxa de acerto no conjunto de treino;
    Y_pred = vSinal(np.matmul(X_tr, a) + b)
    diff = np.subtract(Y_pred, Y_tr.transpose())
    tx_acerto = 1 - np.absolute(diff).sum()/diff.size/2 # taxa de acerto \in [0,1]
    tr[i] = tx_acerto # add resultado no vetor ts
    cm_tr += confusion_matrix(Y_tr, Y_pred)
    
    # Calculando taxa de acerto no conjunto de teste;
    Y_pred = vSinal(np.matmul(X_ts, a) + b)
    diff = np.subtract(Y_pred, Y_ts.transpose())
    tx_acerto = 1 - np.absolute(diff).sum()/diff.size/2 # taxa de acerto \in [0,1]
    ts[i] = tx_acerto # add resultado no vetor ts
    cm_ts += confusion_matrix(Y_ts, Y_pred)

cm_tr/=(200*.8*100) # (número de amostras)*(porcentagem dedicada ao cojunto)*(100 iterações acumuladas)
cm_ts/=(200*.2*100)

CPU times: user 1min 16s, sys: 1min 47s, total: 3min 4s
Wall time: 28.7 s


In [17]:
# Estatísticas do treino/teste
data = np.column_stack((tr, ts))
resultMQL = pd.DataFrame(data, columns=["T.A. no Treino","T.A. no Teste"])
print("Estatísticas do treino e teste do modelo:")
statsMQL = resultMQL.describe()
display(statsMQL)

# Matriz de confusão média. OBS: C[i,j] = pertence a 'i' e foi predita como 'j'. Coluna é predição e linha é o real.
# + taxa de acerto média por classe
temp_tr = np.matrix([ [cm_tr[0,0]/cm_tr[0,:].sum()], [cm_tr[1,1]/cm_tr[1,:].sum()] ])
temp_tr = np.concatenate((cm_tr, temp_tr), axis=1)
temp_ts = np.matrix([ [cm_ts[0,0]/cm_ts[0,:].sum()], [cm_ts[1,1]/cm_ts[1,:].sum()] ])
temp_ts = np.concatenate((cm_ts, temp_ts), axis=1)

cmMQL_tr = pd.DataFrame(temp_tr, columns=["Normal", "Câncer", "T.A. Média"], index=[["Normal", "Câncer"]])
cmMQL_ts = pd.DataFrame(temp_ts, columns=["Normal", "Câncer", "T.A. Média"], index=[["Normal", "Câncer"]])

print("Matriz de confusão mais taxa de acerto média no conjunto de treino:")
display(cmMQL_tr)
print("Matriz de confusão mais taxa de acerto média no conjunto de teste:")
display(cmMQL_ts)

Estatísticas do treino e teste do modelo:


,T.A. no Treino,T.A. no Teste
count,100.0,100.000000
mean,1.0,0.876500
std,0.0,0.053041
min,1.0,0.750000
25%,1.0,0.850000
50%,1.0,0.875000
75%,1.0,0.925000
max,1.0,0.975000


Matriz de confusão mais taxa de acerto média no conjunto de treino:


,Normal,Câncer,T.A. Média
Normal,0.559937,0.000000,1.0
Câncer,0.000000,0.440063,1.0


Matriz de confusão mais taxa de acerto média no conjunto de teste:


,Normal,Câncer,T.A. Média
Normal,0.48225,0.07800,0.860776
Câncer,0.04550,0.39425,0.896532


Concluímos ao ler as Tabelas \ref{tab:statsMQL}, \ref{tab:cmMQL_tr} e \ref{tab:cmMQL_ts} que o conjunto de dados foi facilmente modelado pela regressão linear, obtendo 100% de acerto no conjunto de treino e alta taxa de acerto no conjunto de teste.

<h3>
1.2 Perceptron Simples
</h3>

Para a implementação do perceptron simples uma classe foi criada assim como funções auxiliares.

In [18]:
# Funções de ativação
from numpy import exp

def sinal(x):
    return +1 if x >= 0 else -1

def sigmoid(x):
    return 1/(1+exp(-x))

def tanHiper(x):
    return (1-exp(-x))/(1+exp(-x))

# Versão vetorial das funções:
vSinal    = np.vectorize(sinal)
vSigmoid  = np.vectorize(sigmoid)
vTanHiper = np.vectorize(tanHiper)

In [19]:
# Classe dos perceptrons simples
class PerceptronSimples:
    'Classe criada para descrever as redes de n perceptrons simples em paralelo.'
    
    def __init__(self, nEntradas, nSaidas):
        self.nEntradas = nEntradas
        self.nSaidas   = nSaidas
        self.param = None
        self.trainHist = {
            "train": {"tx": [], "cm": []}, 
            "test":  {"tx": [], "cm": []} 
        } 
        
        
    def initParam(self): # inicialização aleatória dos pesos por distribuição uniforme
        self.param = np.random.rand(self.nEntradas+1, self.nSaidas) # +1 por causa do bias
        # limpeza do histórico de treino
        self.trainHist = {
            "train": {"tx": [], "cm": []}, 
            "test":  {"tx": [], "cm": []} 
        } 
        
    def prever(self, X): # Y = [-1_{nAmostrasx1} X]*w
                         # As amostras na matriz X estão nas linhas, ou deitadas
                         # As variáveis então são as colunas
        XplusBias = np.concatenate((-np.ones((len(X),1)), X), axis=1)
        U = np.matmul(XplusBias, self.param) # ativação
        Y = vSinal(U) # saída do perceptron
        return Y
    
    def treinar(self, X_tr, Y_tr, X_ts, Y_ts, eta, nEpocas):
        for _ in range(nEpocas):
            # Embaralhar vetores de treinamento
            nAmostras = X_tr.shape[0]
            novosIndices = np.arange(nAmostras)
            np.random.shuffle(novosIndices)
            X = X_tr[novosIndices]
            Y = Y_tr[novosIndices]

            # Treino step-by-step
            for i in range(len(X)):
                amostra = X[i,:].reshape(1,len(X[i,:])) # colocando amostra como vetor linha
                Y_pred = self.prever(amostra) 
                E = Y[i,:] - Y_pred # erro da perceptron na amostra em questão
                #print("{}\n{}\n{}\n".format(self.param.shape, E.shape,amostra.transpose().shape))
                self.param += eta*np.concatenate((-np.ones((1,len(amostra))), amostra), axis=1).transpose()*E
                
            self.updateHist(X_tr, Y_tr, X_ts, Y_ts)
    
    def updateHist(self, X_tr, Y_tr, X_ts, Y_ts):
        # Atualizando o histórico de treino da rede
        # conjunto de treino:
        tr_tx, tr_cm = taxaAcerto(Y_tr, self.prever(X_tr))
        self.trainHist["train"]["tx"].append(tr_tx)
        self.trainHist["train"]["cm"].append(tr_cm)
        # conjunto de teste:
        ts_tx, ts_cm = taxaAcerto(Y_ts, self.prever(X_ts))
        self.trainHist["test"]["tx"].append(ts_tx)
        self.trainHist["test"]["cm"].append(ts_cm)
            
    def plotTrainHist(self):
        epocas = [i for i in range(1,1+len(self.trainHist["train"]["tx"]))]

        # Gráfico da taxa de acerto ao longo das épocas
        traceTrTx = go.Scatter(
            x = epocas, 
            y = self.trainHist["train"]["tx"], 
            mode='lines+markers',
            name='conjunto de treino'
        )
        traceTsTx = go.Scatter(
            x = epocas, 
            y = self.trainHist["test"]["tx"], 
            mode='lines+markers',
            name='conjunto de teste'
        )
        data = [traceTrTx, traceTsTx]

        layoutTx = go.Layout(
            title = "Taxas de acerto ao longo do treinamento",
            legend=dict(orientation="h", y=-.05),
            xaxis=dict(title="Época"),
            yaxis=dict(title="Taxa de acerto [0,1]")
        )

        fig = go.Figure(data=data, layout=layoutTx)
        plt.iplot(fig)

        # Gráfico da matriz de confusão ao longo das épocas
        for nome in ["train","test"]:
            vn, fp, fn, vp = [],[],[],[]
            for cm in self.trainHist[nome]["cm"]:
                vn.append(cm[0,0])
                fp.append(cm[0,1])
                fn.append(cm[1,0])
                vp.append(cm[1,1])

            traceVn = go.Scatter(x = epocas, y = vn, mode='lines+markers', name='verdadeiros negativos')
            traceFp = go.Scatter(x = epocas, y = fp, mode='lines+markers', name='falsos positivos')
            traceFn = go.Scatter(x = epocas, y = fn, mode='lines+markers', name='falsos negativos')
            traceVp = go.Scatter(x = epocas, y = vp, mode='lines+markers', name='verdadeiros positivos')

            data = [traceVp, traceVn, traceFp, traceFn]

            layoutTx = go.Layout(
                title = "Matriz de confusão ao longo do treinamento ["+nome+"]",
                legend=dict(orientation="h", y=-.05),
                xaxis=dict(title="Época"),
                yaxis=dict(title="Taxa [0,1]")
            )

            fig = go.Figure(data=data, layout=layoutTx)
            plt.iplot(fig)
            

def taxaAcerto(Y_real, Y_pred):
    diff = Y_pred - Y_real
    tx_acerto = 1 - np.absolute(diff).sum()/diff.size/2 # taxa de acerto \in [0,1]
    cm = confusion_matrix(Y_real, Y_pred)/len(Y_real) # divide-se pelo número de amostras para \in [0,1]
    return (tx_acerto, cm)

Como exemplo de treinamento e convergência as Figura \ref{fig:ps1tx}, \ref{fig:ps1cm_tr} e \ref{fig:ps1cm_ts} foram geradas a partir do código a seguir:

In [20]:
ps = PerceptronSimples(7000,1)
ps.initParam()

X_tr, X_ts, Y_tr, Y_ts = train_test_split(X_v4.values, Y.values, test_size=0.2)
ps.treinar(X_tr, Y_tr, X_ts, Y_ts, eta=.01, nEpocas=10)

ps.plotTrainHist()

Em menos de 10 épocas o perceptron "resolve" o problema, atingindo 100% de acerto no conjunto de treinamento e uma taxa alta de acerto no conjunto de teste. Abaixo foi implementada a análise mais sofisticada:

In [21]:
%%time
ps = PerceptronSimples(7000,1) # 7000 variáveis de entrada

tr = [0]*100 # taxa de acerto no treino
ts = [0]*100 # taxa de acerto no teste
cm_tr = np.zeros((2,2)) # matriz de confusão da média no treino
cm_ts = np.zeros((2,2)) # matriz de confusão da média no teste
for i in range(100):
    # Separação aleatória do dataset em 80% de treino e 20% de teste
    X_tr, X_ts, Y_tr, Y_ts = train_test_split(X_v4.values, Y.values, test_size=0.2)
    
    # Treino do modelo
    ps.initParam()
    ps.treinar(X_tr, Y_tr, X_ts, Y_ts, eta=.01, nEpocas=10)

    # Taxa de acerto no conjunto de treino;
    tr[i] =  ps.trainHist["train"]["tx"][-1]
    cm_tr += ps.trainHist["train"]["cm"][-1]
    
    # Taxa de acerto no conjunto de teste;
    ts[i] =  ps.trainHist["test"]["tx"][-1]
    cm_ts += ps.trainHist["test"]["cm"][-1]

cm_tr/=100 # 100 iterações acumuladas
cm_ts/=100

CPU times: user 1min 22s, sys: 2min 24s, total: 3min 46s
Wall time: 35.5 s


In [22]:
# Estatísticas do treino/teste
data = np.column_stack((tr, ts))
resultPS = pd.DataFrame(data, columns=["T.A. no Treino","T.A. no Teste"])
print("Estatísticas do treino e teste do PS:")
statsPS = resultPS.describe()
display(statsPS)

# Matriz de confusão média. OBS: C[i,j] = pertence a 'i' e foi predita como 'j'. Coluna é predição e linha é o real.
# + taxa de acerto média por classe
temp_tr = np.matrix([ [cm_tr[0,0]/cm_tr[0,:].sum()], [cm_tr[1,1]/cm_tr[1,:].sum()] ])
temp_tr = np.concatenate((cm_tr, temp_tr), axis=1)
temp_ts = np.matrix([ [cm_ts[0,0]/cm_ts[0,:].sum()], [cm_ts[1,1]/cm_ts[1,:].sum()] ])
temp_ts = np.concatenate((cm_ts, temp_ts), axis=1)

cmPS_tr = pd.DataFrame(temp_tr, columns=["Normal", "Câncer", "T.A. Média"], index=[["Normal", "Câncer"]])
cmPS_ts = pd.DataFrame(temp_ts, columns=["Normal", "Câncer", "T.A. Média"], index=[["Normal", "Câncer"]])

print("Matriz de confusão mais taxa de acerto média no conjunto de treino do PS:")
display(cmPS_tr)
print("Matriz de confusão mais taxa de acerto média no conjunto de teste do PS:")
display(cmPS_ts)

Estatísticas do treino e teste do PS:


,T.A. no Treino,T.A. no Teste
count,100.0,100.000000
mean,1.0,0.701750
std,0.0,0.072497
min,1.0,0.500000
25%,1.0,0.650000
50%,1.0,0.700000
75%,1.0,0.750000
max,1.0,0.875000


Matriz de confusão mais taxa de acerto média no conjunto de treino do PS:


,Normal,Câncer,T.A. Média
Normal,0.560125,0.000000,1.0
Câncer,0.000000,0.439875,1.0


Matriz de confusão mais taxa de acerto média no conjunto de teste do PS:


,Normal,Câncer,T.A. Média
Normal,0.33850,0.22100,0.605004
Câncer,0.07725,0.36325,0.824631


Apresentando performance parecida com o MQL o PS só teve resultados piores nos conjuntos de teste. Nas matrizes de confusão vemos uma história parecida, em que no conjunto de treino a taxa de acerto média por classe é 100% e no conjunto de teste o modelo aparenta ter maior facilidade em acertar o diagnóstico quando o paciente efetivamente tem câncer.

<h3>
1.3 Perceptron Multicamadas
</h3>

Analogamente ao PS uma classe foi desenvolvida. Na definição da classe abaixo foi escolhido como função de ativação para as camadas oculta e de saída a tangente hiperbólica. Também foi adotado o método do gradiente descendente para treino da rede e uma atualização dos pesos \textit{step-by-step}, ou seja, os parâmetros da rede são atualizados a cada amostra.

In [23]:
# Classe dos Perceptrons Multicamadas
class PerceptronMulticamada(PerceptronSimples):
    def __init__(self, nEntradas, nOcultas, nSaidas):
        PerceptronSimples.__init__(self, nEntradas, nSaidas)
        self.nOcultas = nOcultas
                
    def initParam(self): # inicialização aleatória dos pesos por distribuição uniforme
        self.param = [2*np.random.rand(self.nEntradas+1, self.nOcultas)-1,
                      2*np.random.rand(self.nOcultas+1, self.nSaidas)-1]   # +1 por causa do bias
        # limpeza do histórico de treino
        self.trainHist = {"train": {"tx": [], "cm": []}, "test":  {"tx": [], "cm": []}} 
        
    def prever(self, X): # Y = [-1_{nAmostrasx1} X]*w
                         # As amostras na matriz X estão nas linhas, ou deitadas
                         # As variáveis então são as colunas
        XplusBias = np.concatenate((-np.ones((len(X),1)), X), axis=1)
        U1 = np.matmul(XplusBias, self.param[0]) # ativação da primeira camada
        Y1 = vTanHiper(U1)                       # saída da camada oculta
        
        XplusBias = np.concatenate((-np.ones((len(Y1),1)), Y1), axis=1)
        U2 = np.matmul(XplusBias, self.param[1]) # ativação da camada de saída
        Y2 = vTanHiper(U2)                       # saída da MLP
        return Y2, Y1
    
    def treinar(self, X_tr, Y_tr, X_ts, Y_ts, eta, nEpocas):
        for e in range(nEpocas):
            # Embaralhar vetores de treinamento
            novosIndices = np.arange(X_tr.shape[0])
            np.random.shuffle(novosIndices)
            X, Y = X_tr[novosIndices], Y_tr[novosIndices]

            # Treino step-by-step
            for i in range(len(X)):
                amostra = X[i,:].reshape(1,len(X[i,:])) # colocando amostra como vetor linha
                Y_pred, H_pred = self.prever(amostra)
                
                # Cálculo do erro e gradientes
                E = Y[i,:] - Y_pred # erro da MLP na amostra em questão
                #print(Y_pred.shape, E.shape)
                
                Dk = 0.5*(1 - np.power(Y_pred, 2)) # derivada da tangente hiperbólica (camada de saida)
                DDk = E*Dk               # gradiente local (camada de saida)
                
                Di = 0.5*(1 - np.power(H_pred,2).transpose())     # derivada da sigmoide logistica (camada oculta)
                DDi = np.multiply(Di, np.matmul(self.param[1][1:, :], DDk)) # gradiente local (camada oculta)
                
                # Atualização dos pesos
                self.param[1] += eta*DDk*np.concatenate((-np.ones((1,len(H_pred))), H_pred), axis=1).transpose()
                self.param[0] += eta*np.concatenate((-np.ones((1,len(amostra))), amostra), axis=1).transpose()*DDi.transpose()
                
            # Atualizando histórico
            self.updateHist(X_tr, Y_tr, X_ts, Y_ts)

def taxaAcerto(Y_real, Y_pred):
    Y_pred = vSinal(Y_pred[0])
    diff = Y_pred - Y_real
    tx_acerto = 1 - np.absolute(diff).sum()/diff.size/2 # taxa de acerto \in [0,1]
    cm = confusion_matrix(Y_real, Y_pred)/len(Y_real) # divide-se pelo número de amostras para \in [0,1]
    return (tx_acerto, cm)

Como exemplo da classe em funcionamento as Figuras \ref{fig:mlp1tx}, \ref{fig:mlp1cm_tr} e \ref{fig:mlp1cm_ts} foram geradas utilizando o código abaixo. Obs: o número de neurônios ocultos foi escolhido arbitrariamente.

In [26]:
%%time
# Separação aleatória do dataset em 80% de treino e 20% de teste
X_tr, X_ts, Y_tr, Y_ts = train_test_split(X_v4.values, Y.values, test_size=0.2)

# Construção e treino do modelo
mlp = PerceptronMulticamada(7000, 5, 1)
mlp.initParam()
mlp.treinar(X_tr, Y_tr, X_ts, Y_ts, eta=.05, nEpocas=25)
mlp.plotTrainHist()

CPU times: user 9.3 s, sys: 18 s, total: 27.3 s
Wall time: 4.56 s


O grande problema de redes MLP é identificar quantas camadas internas e quantos neurônios cada camada interna deve ter. Para resolver esse problema, com o poder computacional disponível, foi decidido utilizar apenas uma camada oculta, visto que o perceptron simples já teve bons resultados, e uma procura em grade (\textit{Grid Search})  para tentar achar a melhor quantidade de neurônios ocultos.

A escolha da arquitetura da rede foi feita baseada na mesma métrica, ou função objetivo, apresentada anteriormente:

$$\text{maximizar } f_0(ts)=\mathbb{E}\{t_s\} - \gamma\sigma\{t_s\}$$

Onde $t_s$ é a variável aleatória que representa a taxa de acerto da rede no conjunto de teste. Para essa busca o valor de $\gamma$ adotado também foi 2.

Os possíveis valores de neurônios da camada oculta são apresentados a seguir, havendo 20 possibilidades:

In [27]:
arq = [i for i in range(2,500, 25)]
print("Possíveis quantidades de neurônios ocultos: {}.".format(arq))

Possíveis quantidades de neurônios ocultos: [2, 27, 52, 77, 102, 127, 152, 177, 202, 227, 252, 277, 302, 327, 352, 377, 402, 427, 452, 477].


O código a seguir implementa a busca em grade pela melhor arquitetura segundo a função objetivo escolhida:

In [28]:
resultMLP = pd.read_csv('resultMLP.csv')
resultMLP = resultMLP.sort_values("f_0(ts)", ascending=False)
resultMLP

,Nº hidden neurons,Mean(ts),Std(ts),f_0(ts)
2,52,0.69375,0.066085,0.561581
7,177,0.69300,0.067089,0.558821
10,252,0.70375,0.075529,0.552691
12,302,0.70000,0.074498,0.551003
16,402,0.70400,0.077356,0.549287
19,477,0.70175,0.076751,0.548248
8,202,0.70100,0.076642,0.547716
1,27,0.69200,0.073133,0.545733
5,127,0.70125,0.078928,0.543393
3,77,0.68975,0.073832,0.542086


De acordo com os resultados da Tabela \ref{tab:resultMLP} a MLP mais performante foi a com 52 neurônios ocultos, portanto essa arquitetura foi adotada. A avaliação final do modelo foi realizada:

In [29]:
%%time
mlp = PerceptronMulticamada(7000, 52, 1)

tr = [0]*100 # taxa de acerto no treino
ts = [0]*100 # taxa de acerto no teste
cm_tr = np.zeros((2,2)) # matriz de confusão da média no treino
cm_ts = np.zeros((2,2)) # matriz de confusão da média no teste
for i in range(100):
    # Separação aleatória do dataset em 80% de treino e 20% de teste
    X_tr, X_ts, Y_tr, Y_ts = train_test_split(X_v4.values, Y.values, test_size=0.2)
    
    # Treino do modelo
    mlp.initParam()
    mlp.treinar(X_tr, Y_tr, X_ts, Y_ts, eta=.1, nEpocas=20)

    # Taxa de acerto no conjunto de treino;
    tr[i] =  mlp.trainHist["train"]["tx"][-1]
    cm_tr += mlp.trainHist["train"]["cm"][-1]
    
    # Taxa de acerto no conjunto de teste;
    ts[i] =  mlp.trainHist["test"]["tx"][-1]
    cm_ts += mlp.trainHist["test"]["cm"][-1]

cm_tr/=100 # 100 iterações acumuladas
cm_ts/=100

/home/romulo/Dropbox/Mestrado/[00 old]/[DISCIPLINA] Inteligência Computacional Aplicada/venv/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning:

overflow encountered in exp

/home/romulo/Dropbox/Mestrado/[00 old]/[DISCIPLINA] Inteligência Computacional Aplicada/venv/lib/python3.6/site-packages/ipykernel_launcher.py:11: RuntimeWarning:

invalid value encountered in double_scalars



CPU times: user 43min 51s, sys: 1h 8min 39s, total: 1h 52min 31s
Wall time: 16min 36s


In [30]:
# Estatísticas do treino/teste
data = np.column_stack((tr, ts))
resultMLP = pd.DataFrame(data, columns=["T.A. no Treino","T.A. no Teste"])
print("Estatísticas do treino e teste do MLP:")
statsMLP = resultMLP.describe()
display(statsMLP)

# Matriz de confusão média. OBS: C[i,j] = pertence a 'i' e foi predita como 'j'. Coluna é predição e linha é o real.
# + taxa de acerto média por classe
temp_tr = np.matrix([ [cm_tr[0,0]/cm_tr[0,:].sum()], [cm_tr[1,1]/cm_tr[1,:].sum()] ])
temp_tr = np.concatenate((cm_tr, temp_tr), axis=1)
temp_ts = np.matrix([ [cm_ts[0,0]/cm_ts[0,:].sum()], [cm_ts[1,1]/cm_ts[1,:].sum()] ])
temp_ts = np.concatenate((cm_ts, temp_ts), axis=1)

cmMLP_tr = pd.DataFrame(temp_tr, columns=["Normal", "Câncer", "T.A. Média"], index=[["Normal", "Câncer"]])
cmMLP_ts = pd.DataFrame(temp_ts, columns=["Normal", "Câncer", "T.A. Média"], index=[["Normal", "Câncer"]])

print("Matriz de confusão mais taxa de acerto média no conjunto de treino do MLP:")
display(cmMLP_tr)
print("Matriz de confusão mais taxa de acerto média no conjunto de teste do MLP:")
display(cmMLP_ts)

Estatísticas do treino e teste do MLP:


,T.A. no Treino,T.A. no Teste
count,100.000000,100.00000
mean,0.975563,0.67950
std,0.048094,0.09006
min,0.531250,0.47500
25%,0.975000,0.60000
50%,0.981250,0.67500
75%,0.993750,0.75000
max,1.000000,0.85000


Matriz de confusão mais taxa de acerto média no conjunto de treino do MLP:


,Normal,Câncer,T.A. Média
Normal,0.548875,0.010937,0.980462
Câncer,0.013500,0.426687,0.969331


Matriz de confusão mais taxa de acerto média no conjunto de teste do MLP:


,Normal,Câncer,T.A. Média
Normal,0.39050,0.17025,0.696389
Câncer,0.15025,0.28900,0.657940


De forma geral o modelo MLP apresentou piores resultados que os outros modelos.

<h3>
1.4 Comparação dos modelos estudados
</h3>

In [31]:
statsAll = pd.DataFrame(np.concatenate((statsMQL.values, statsPS.values, statsMLP.values), axis=1), 
             columns = ["MQL_tr", "MQL_ts", "PS_tr", "PS_ts", "MLP_tr", "MLP_ts"],
             index=statsMQL.index)
print("Estatísticas de treino/teste dos 3 modelos:")
statsAll

Estatísticas de treino/teste dos 3 modelos:


,MQL_tr,MQL_ts,PS_tr,PS_ts,MLP_tr,MLP_ts
count,100.0,100.000000,100.0,100.000000,100.000000,100.00000
mean,1.0,0.876500,1.0,0.701750,0.975563,0.67950
std,0.0,0.053041,0.0,0.072497,0.048094,0.09006
min,1.0,0.750000,1.0,0.500000,0.531250,0.47500
25%,1.0,0.850000,1.0,0.650000,0.975000,0.60000
50%,1.0,0.875000,1.0,0.700000,0.981250,0.67500
75%,1.0,0.925000,1.0,0.750000,0.993750,0.75000
max,1.0,0.975000,1.0,0.875000,1.000000,0.85000


A Tabela \ref{tab:statsAll} aponta que quanto mais complexo for o modelo pior seu desempenho nos conjuntos de teste. Enquanto todos os modelos aparentaram ser poderosos o suficiente para descrever os conjuntos de treino (com exceção de algumas amostras da MLP que apresentaram resultados baixos) eles não tiverem desempenho equivalente nos conjuntos de teste. Terminamos esse trecho do trabalho levantando algumas hipóteses para explicar esses resultados, hipóteses que poderão ser estudadas em trabalhos futuros:

* Os dados não tem grande complexidade, portanto modelos mais complexos como MLP são mais susceptíveis a \textit{overfitting};
* A regra de treino e/ou os iperparâmetros do PS e MLP não foram bem ajustados para o problema.

<h2>
2. Questão 02
</h2>

Primeiro o conjunto de dados \textit{bodyfat} foi carregado e normalizado (como no Trabalho Computacional 01):

In [32]:
import pandas as pd
import numpy as np

labels = ["Density", "Percent body fat", "Age (years)", "Weight (lbs)", "Height (inches)", "Neck circum. (cm)",
          "Chest circum. (cm)", "Abdomen circum. (cm)", "Hip circum. (cm)", "Thigh circum. (cm)", "Knee circum. (cm)",
          "Ankle circum. (cm)", "Biceps circum. (cm)","Forearm circum. (cm)","Wrist circum. (cm)"]
dataset = pd.read_csv('dataset_bodyfat.txt', delim_whitespace=True, names = labels)

y_label = ["Percent body fat"]
y_data = dataset[y_label]

x_label = labels[2:] # excluindo densidade corporal
x_data = dataset[x_label]

# Normalização das variáveis de entrada:
x_normal = (x_data - x_data.mean()) / x_data.std()

Um classe para o modelo ADALINE foi desenvolvida para facilitar a análise:

In [33]:
# Classe ADALINE, herda de PS
class ADALINE(PerceptronSimples):    
    def __init__(self, nEntradas, nSaidas):
        self.nEntradas = nEntradas
        self.nSaidas   = nSaidas
        self.param = None
        self.trainHist = { # armazena o Erro Quadrático Médio (MSE)
            "train": [], "test":  []
        }
        
    def initParam(self): # inicialização aleatória dos pesos por distribuição uniforme
        self.param = np.random.rand(self.nEntradas+1, self.nSaidas) # +1 por causa do bias
        # limpeza do histórico de treino
        self.trainHist = {"train": [], "test":  []}
    
    def prever(self, X):
        XplusBias = np.concatenate((-np.ones((len(X),1)), X), axis=1)
        U = np.matmul(XplusBias, self.param) # ativação
        Y = U # função de ativação = identidade, 
              # visto que se trata de um problema de regressão
        return Y
    
    def updateHist(self, X_tr, Y_tr, X_ts, Y_ts):
        # Atualizando o histórico de treino da rede
        self.trainHist["train"].append( self.mse(Y_tr, self.prever(X_tr)) )
        self.trainHist["test"].append( self.mse(Y_ts, self.prever(X_ts)) )
        
            
    def plotTrainHist(self):
        epocas = [i for i in range(1,1+len(self.trainHist["train"]))]

        # Gráfico do MSE ao longo das épocas
        traceTrMSE = go.Scatter(
            x = epocas, 
            y = self.trainHist["train"], 
            mode='lines+markers',
            name='conjunto de treino'
        )
        traceTsMSE = go.Scatter(
            x = epocas, 
            y = self.trainHist["test"], 
            mode='lines+markers',
            name='conjunto de teste'
        )
        data = [traceTrMSE, traceTsMSE]

        layoutMSE = go.Layout(
            title = "MSE ao longo do treinamento",
            legend=dict(orientation="h", y=-.05),
            xaxis=dict(title="Época"),
            yaxis=dict(title="Erro Quadrático Médio (MSE)")
        )

        fig = go.Figure(data=data, layout=layoutMSE)
        plt.iplot(fig)

    @classmethod
    def mse(self, y_real, y_pred):
        e_vec = y_real - y_pred
        #print(e_vec)
        MSE = np.dot(np.transpose(e_vec), e_vec) / (2*len(e_vec))
        #print(np.transpose(e_vec).shape, e_vec.shape)
        #print(MSE)
        return MSE[0][0]

Como exemplo da classe ADALINE em funcionamento a Figura \ref{fig:ada1mse} foi gerada utilizando o código abaixo:

In [34]:
ada = ADALINE(x_data.values.shape[1],1)
ada.initParam()

x = x_normal
X_tr, X_ts, Y_tr, Y_ts = train_test_split(x.values, y_data.values, test_size=0.2)

ada.treinar(X_tr, Y_tr, X_ts, Y_ts, eta=1e-3, nEpocas=100)
ada.plotTrainHist()

Uma nova classe, \textit{MLP_reg}, foi criada para lidar com problema de regressão apresentado:

In [35]:
# Classe dos Perceptrons Multicamadas
class MLP_reg(ADALINE):
    def __init__(self, nEntradas, nOcultas, nSaidas):
        PerceptronSimples.__init__(self, nEntradas, nSaidas)
        self.nOcultas = nOcultas
    
    def initParam(self): # inicialização aleatória dos pesos por distribuição uniforme
        self.param = [2*np.random.rand(self.nEntradas+1, self.nOcultas)-1,
                      2*np.random.rand(self.nOcultas+1, self.nSaidas)-1]   # +1 por causa do bias
        # limpeza do histórico de treino
        self.trainHist = {"train": [], "test": []}
        
    def prever(self, X): 
        XplusBias = np.concatenate((-np.ones((len(X),1)), X), axis=1)
        U1 = np.matmul(XplusBias, self.param[0]) # ativação da primeira camada
        Y1 = vTanHiper(U1)                       # saída da camada oculta
        
        XplusBias = np.concatenate((-np.ones((len(Y1),1)), Y1), axis=1)
        U2 = np.matmul(XplusBias, self.param[1]) # ativação da camada de saída
        Y2 = U2                       # saída da MLP como identidade
        return Y2, Y1
    
    def treinar(self, X_tr, Y_tr, X_ts, Y_ts, eta, nEpocas):
        for e in range(nEpocas):
            # Embaralhar vetores de treinamento
            novosIndices = np.arange(X_tr.shape[0])
            np.random.shuffle(novosIndices)
            X, Y = X_tr[novosIndices], Y_tr[novosIndices]
            
            # Treino step-by-step
            for i in range(len(X)):
                amostra = X[i,:].reshape(1,len(X[i,:])) # colocando amostra como vetor linha
                Y_pred, H_pred = self.prever(amostra)
                
                # Cálculo do erro e gradientes
                E = Y[i,:] - Y_pred # erro da MLP na amostra em questão
                #print(Y_pred.shape, E.shape)
                
                #Dk = 0.5*(1 - np.power(Y_pred, 2)) # derivada da tangente hiperbólica (camada de saida)
                Dk = 1 # derivada da identidade sempre é 1
                DDk = E*Dk               # gradiente local (camada de saida)
                
                Di = 0.5*(1 - np.power(H_pred,2).transpose())     # derivada da sigmoide logistica (camada oculta)
                DDi = np.multiply(Di, np.matmul(self.param[1][1:, :], DDk)) # gradiente local (camada oculta)
                
                # Atualização dos pesos
                self.param[1] += eta*DDk*np.concatenate((-np.ones((1,len(H_pred))), H_pred), axis=1).transpose()
                self.param[0] += eta*np.concatenate((-np.ones((1,len(amostra))), amostra), axis=1).transpose()*DDi.transpose()
                
            # Atualizando histórico
            self.updateHist(X_tr, Y_tr, X_ts, Y_ts)

    @classmethod
    def mse(self, y_real, y_pred):
        e_vec = y_real - y_pred[0]
        #print(e_vec)
        MSE = np.dot(np.transpose(e_vec), e_vec) / (2*len(e_vec))
        #print(np.transpose(e_vec).shape, e_vec.shape)
        #print(MSE)
        return MSE[0][0]

Como exemplo da classe \textit{MLP_reg} em funcionamento a Figura \ref{fig:mlp1mse} foi gerada utilizando o código abaixo:

In [36]:
%%time
X_tr, X_ts, Y_tr, Y_ts = train_test_split(x.values, y_data.values, test_size=0.2)
mlp = MLP_reg(13, 10, 1)
mlp.initParam()
mlp.treinar(X_tr, Y_tr, X_ts, Y_ts, eta=1e-3, nEpocas=100)
mlp.plotTrainHist()

CPU times: user 3.37 s, sys: 260 µs, total: 3.37 s
Wall time: 3.36 s


Para o modelo MLP foi adotada apenas uma camada oculta e seu número de neurônios foi escolhida após uma busca em grade com a  função objetivo:

$$\text{minimizar} \hspace{.3cm} f_0(mse)=\mathbb{E}\{mse\} + 2\sigma\{mse\}$$

Onde $mse$ é a variável aleatória que representa o erro quadrático médio calculado a cada \textit{resampling}. Os possíveis número de neurônios ocultos avaliados foram:

In [37]:
arq = [i for i in range(2,26)]
print(arq)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]


O código abaixo implementou a busca em grade proposta:

In [38]:
resultMLP_reg = pd.read_csv('resultMLP_reg.csv')
resultMLP_reg = resultMLP_reg.sort_values("f_0(mse_ts)", ascending=True)
resultMLP_reg

,Nº hidden neurons,Mean(mse_ts),Std(mse_ts),f_0(mse_ts)
0,2,11.878188,2.146518,16.171225
6,20,12.293755,2.147947,16.589649
8,18,12.306348,2.159851,16.626050
3,22,12.247404,2.357993,16.963391
12,25,12.495701,2.261248,17.018197
5,11,12.290537,2.379235,17.049007
1,24,12.146107,2.465390,17.076887
2,16,12.222160,2.483872,17.189905
11,23,12.493476,2.368493,17.230461
4,13,12.265455,2.497431,17.260317


De acordo com os resultados da Tabela \ref{tab:resultMLP_reg} a arquitetura mais performante foi MLP(13,2,1), ela foi portanto adotada.

Todos os modelos foram avaliados no conjunto de dados e a métrica de comparação foi o Erro Quadrático Médio (em inglês a sigla \textit{MSE}). Foram salvos o MSE de treino e teste para cada uma das cem rodadas, os resultados seguem abaixo:

In [39]:
%%time
from sklearn import linear_model

# Inicialização dos objetos
mql = linear_model.LinearRegression()
ada = ADALINE(13, 1)
mlp = MLP_reg(13, 2, 1)

MQL_tr, MQL_ts, ADALINE_tr, ADALINE_ts, MLP_tr, MLP_ts = 0,1,2,3,4,5
data = np.zeros((100,6))
for i in range(100):
    # Separação aleatória do dataset em 80% de treino e 20% de teste
    X_tr, X_ts, Y_tr, Y_ts = train_test_split(x.values, y_data.values, test_size=0.2)
    
    # Otimização dos modelos
    mql.fit(X_tr, Y_tr)
    
    ada.initParam()
    ada.treinar(X_tr, Y_tr, X_ts, Y_ts, eta=1e-3, nEpocas=100)
    
    mlp.initParam()
    mlp.treinar(X_tr, Y_tr, X_ts, Y_ts, eta=1e-3, nEpocas=100)

    # Salvando resultados:
    # Usando o método da classe ADALINE no MQL porque ele se aplica da 
    # mesma forma
    data[i, MQL_tr] = ADALINE.mse(Y_tr, mql.predict(X_tr))
    data[i, MQL_ts] = ADALINE.mse(Y_ts, mql.predict(X_ts))
    
    data[i, ADALINE_tr] = ada.trainHist["train"][-1]
    data[i, ADALINE_ts] = ada.trainHist["test"][-1]
    
    data[i, MLP_tr] = mlp.trainHist["train"][-1]
    data[i, MLP_ts] = mlp.trainHist["test"][-1]

CPU times: user 4min 54s, sys: 0 ns, total: 4min 54s
Wall time: 4min 54s


In [40]:
df_mse = pd.DataFrame(data, columns=["MQL_tr", "MQL_ts", "ADALINE_tr", "ADALINE_ts", "MLP_tr", "MLP_ts"])
print("MSE para conjuntos de treino e teste dos 3 modelos:")
df_mse.describe()

MSE para conjuntos de treino e teste dos 3 modelos:


,MQL_tr,MQL_ts,ADALINE_tr,ADALINE_ts,MLP_tr,MLP_ts
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,8.630998,10.047770,8.898741,10.352849,8.660898,11.839583
std,0.339083,1.696815,0.358581,1.892583,0.644774,2.503780
min,7.826203,6.170501,8.052136,6.217921,7.514315,6.658301
25%,8.373795,9.199920,8.630063,9.236489,8.251758,10.138697
50%,8.643305,9.836658,8.937689,10.271642,8.607992,11.323566
75%,8.807980,10.873656,9.084268,11.372601,8.987635,13.709494
max,9.463881,16.218675,9.812311,16.773660,11.709247,18.553054


A Tabela \ref{tab:df_mse} indica que os 3 modelos utilizados não tiveram diferença significativa entre si quanto à média do MSE, já o modelo MLP apresentou mais instabilidade (valores de desvio-padrão mais altos). Para investigar quão diferentes são os desempenhos dos modelos, do ponto de vista estatístico, o teste de Kolmogorov-Smirnov foi aplicado entre as distribuições de MSE dos conjuntos de teste.

In [41]:
from scipy import stats

newData = np.zeros((3,3))
for linha in range(3):
    for coluna in range(3):
        newData[linha, coluna] = stats.ks_2samp(data[:, 1+linha*2], data[:, 1+coluna*2]).pvalue
        
df_KS = pd.DataFrame(newData, columns=["MQL_ts", "ADALINE_ts", "MLP_ts"],
                              index =[["MQL_ts", "ADALINE_ts", "MLP_ts"]])
print("P-values:")
df_KS

P-values:


,MQL_ts,ADALINE_ts,MLP_ts
MQL_ts,1.000000,0.260553,0.000006
ADALINE_ts,0.260553,1.000000,0.001785
MLP_ts,0.000006,0.001785,1.000000


Concluímos que o MQL e o ADALINE são equivalentes estatisticamente. No caso do MLP os p-values tiveram ordem de grandeza $10^{-4}$, rejeitou-se em vista disso a hipótese nula.

A última análise aplicada aos modelos foi gerar o histograma dos resíduos e realizar o teste de normalidade de Shapiro-Wilk, o trecho de código a seguir implementa essa função:

In [43]:
def plotHistogram(e_vec, nome):
    # Histograma dos resíduos
    data = [go.Histogram(
        x=e_vec,
        histnorm="probability density",
    )]

    titlefont=dict(family='Courier New, monospace', size=18, color='#7f7f7f')
    layoutHist = go.Layout(
        title='Histograma dos resíduos ['+nome+']',
        xaxis=dict(title='Valor do erro', titlefont=titlefont),
        yaxis=dict(title='Densidade de probabilidade', titlefont=titlefont)
    )

    fig = go.Figure(data=data, layout=layoutHist)
    plt.iplot(fig)

# Aplicando otimização dos parâmetros com todos os dados em mãos
x, y = x_normal.values, y_data.values

mql.fit(x, y)
ada.initParam(); ada.treinar(x, y, x, y, eta=1e-3, nEpocas=100)
mlp.initParam(); mlp.treinar(x, y, x, y, eta=1e-3, nEpocas=100)

# Cálculo dos erros e teste de Shapiro-Wilk
e_mql = (y-mql.predict(x)).T.tolist()[0]
e_ada = (y-ada.prever(x)).T.tolist()[0]
e_mlp = (y-mlp.prever(x)[0]).T.tolist()[0]

print("SW P-value dos resíduos de:\
\nMQL:     {mql}\nADALINE: {ada}\nMLP:     {mlp}".format(
    mql=stats.shapiro(e_mql)[1], 
    ada=stats.shapiro(e_ada)[1], 
    mlp=stats.shapiro(e_mlp)[1]))

# Plot dos histogramas
plotHistogram(e_mql, "MQL")
plotHistogram(e_ada, "ADALINE")
plotHistogram(e_mlp, "MLP")

SW P-value dos resíduos de:
MQL:     0.21762634813785553
ADALINE: 0.2556420564651489
MLP:     0.5343580842018127


Pelos p-values apresentados acima e pelas Figuras \ref{fig:histMQL}, \ref{fig:histADALINE} e \ref{fig:histMLP} podemos concluir que os resíduos dos três modelos apresentaram distribuição gaussiana.